In [1]:
from parase_input_package.generate_output import *
from parase_input_package.parase_input import *
rows,columns,bend_penalty,via_penalty,layer1_gird,layer2_grid = parse_gridfile('../benchmark/bench1.grid')
netlist_file_path = '../benchmark/bench1.nl'


50 50 5 1
(50, 50)
(50, 50)


In [2]:
nets,net_num = parse_netlist(netlist_file_path)

In [3]:
print(nets[0:3])


[{'net_id': 1, 'pin1': {'x': 3, 'y': 36, 'layer': 1}, 'pin2': {'x': 3, 'y': 29, 'layer': 1}}, {'net_id': 2, 'pin1': {'x': 4, 'y': 36, 'layer': 1}, 'pin2': {'x': 4, 'y': 29, 'layer': 1}}, {'net_id': 3, 'pin1': {'x': 5, 'y': 36, 'layer': 1}, 'pin2': {'x': 6, 'y': 29, 'layer': 1}}]


In [8]:
def unit_cost_router(rows, columns, layer1_grid, nets):
    routing_table = {}
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        layer1_grid[pin1['x']][pin1['y']] = -1
        layer1_grid[pin2['x']][pin2['y']] = -1
        ## 防止布线在后续的pin上，先将所有的pin标记为-1；
        
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        layer1_grid[pin1['x']][pin1['y']] = 1
        layer1_grid[pin2['x']][pin2['y']] = 1
        path = expand_source_to_target(rows, columns, layer1_grid, pin1, pin2)
        mark_path_on_grid(layer1_grid,path)
        routing_table[net_id] = path
    return routing_table

from queue import Queue

def expand_source_to_target(rows, columns, layer1_grid, source, target):
    queue = Queue()
    visited = set()
    parents = {}

    source_tuple = (source['x'], source['y'], source['layer'])
    target_tuple = (target['x'], target['y'], target['layer'])
    
    queue.put(source_tuple)
    visited.add(source_tuple)

    while not queue.empty():
        current_cell = queue.get()

        if current_cell == target_tuple:
            path = reconstruct_path(source_tuple, target_tuple, parents)
            return path

        neighbors = get_neighbors(rows, columns, current_cell)

        for neighbor in neighbors:
            neighbor_tuple = (neighbor['x'], neighbor['y'], neighbor['layer'])
            if neighbor_tuple not in visited:
                if layer1_grid[neighbor['x']][neighbor['y']] != -1:
                    queue.put(neighbor_tuple)
                    visited.add(neighbor_tuple)
                    parents[neighbor_tuple] = current_cell

    return []

def get_neighbors(rows, columns, cell):
    x, y, layer = cell
    neighbors = []
    if x > 0:
        neighbors.append({'x': x - 1, 'y': y, 'layer': layer})
    if x < columns - 1:
        neighbors.append({'x': x + 1, 'y': y, 'layer': layer})
    if y > 0:
        neighbors.append({'x': x, 'y': y - 1, 'layer': layer})
    if y < rows - 1:
        neighbors.append({'x': x, 'y': y + 1, 'layer': layer})

    return neighbors


def reconstruct_path(source, target, parents):
    path = []
    current = target
    while current != source:
        path.append(current)
        current = parents[current]
    path.append(source)
    path.reverse()
    return path

def mark_path_on_grid(layer1_grid, path):
    for cell in path:
        x, y, _ = cell
        layer1_grid[x][y] = -1

In [9]:
routing_table = unit_cost_router(rows, columns, layer1_gird, nets)

In [10]:
print(routing_table)
filepath_out='../output/out_corerouter_v1'
generate_output_file(filepath_out, net_num+1, routing_table)

{1: [(3, 36, 1), (2, 36, 1), (2, 35, 1), (2, 34, 1), (2, 33, 1), (2, 32, 1), (2, 31, 1), (2, 30, 1), (2, 29, 1), (3, 29, 1)], 2: [(4, 36, 1), (4, 37, 1), (3, 37, 1), (2, 37, 1), (1, 37, 1), (1, 36, 1), (1, 35, 1), (1, 34, 1), (1, 33, 1), (1, 32, 1), (1, 31, 1), (1, 30, 1), (1, 29, 1), (1, 28, 1), (2, 28, 1), (3, 28, 1), (4, 28, 1), (4, 29, 1)], 3: [(5, 36, 1), (5, 35, 1), (5, 34, 1), (5, 33, 1), (5, 32, 1), (5, 31, 1), (5, 30, 1), (5, 29, 1), (6, 29, 1)], 4: [(7, 36, 1), (7, 35, 1), (8, 35, 1), (8, 34, 1), (8, 33, 1), (8, 32, 1), (8, 31, 1), (8, 30, 1), (8, 29, 1), (9, 29, 1)], 5: [(10, 36, 1), (10, 35, 1), (11, 35, 1), (12, 35, 1), (12, 34, 1), (12, 33, 1), (12, 32, 1), (12, 31, 1), (12, 30, 1), (12, 29, 1), (13, 29, 1)], 6: [(49, 36, 1), (49, 37, 1), (48, 37, 1), (47, 37, 1), (46, 37, 1), (45, 37, 1), (44, 37, 1), (43, 37, 1), (42, 37, 1), (41, 37, 1), (40, 37, 1), (39, 37, 1), (38, 37, 1), (37, 37, 1), (36, 37, 1), (36, 36, 1)], 7: [], 8: [(49, 34, 1), (49, 33, 1), (48, 33, 1), (47,